In [1]:
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
# set up splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/sathonphowborom/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


In [3]:
# visit web page
page_url = "https://www.federalreserve.gov/newsevents/pressreleases.htm"
browser.visit(page_url)
time.sleep(10)

press_release_list = []

# iterate through all pages
for i in range(10):
    
    # create html object and scrape into soup
    html = browser.html
    soup = bs(html, "html.parser")

    press_releases = soup.find_all("div", "col-xs-9 col-md-10 eventlist__event")

    for release in press_releases:

        release_url = 'https://www.federalreserve.gov' + release.a['href']
        title = release.text.strip()
        category = release.b.text.strip()

        release_dict = {"url": release_url,
                        "title": title,
                        "category": category
                       }

        press_release_list.append(release_dict)
    
    # click the 'Next' button on each page
    try:
        browser.links.find_by_partial_text('Next').click()
          
    except:
        print("scraping complete")

In [4]:
len(press_release_list)

200

In [6]:
press_release_details = []

for i in range(len(press_release_list)):
    link = press_release_list[i]["url"]
    
    # visit web page
    browser.visit(link)
    time.sleep(1)

    # create html object and scrape into soup
    html = browser.html
    soup = bs(html, "html.parser")
    
    release_date = soup.find("div", class_="heading col-xs-12 col-sm-8 col-md-8").find("p", class_="article__time").text.strip()
    release_time = soup.find("div", class_="heading col-xs-12 col-sm-8 col-md-8").find("p", class_="releaseTime").text.strip()
    
    details_dict = {"url": link,
                    "date": release_date,
                    "time": release_time
                   }
    
    press_release_details.append(details_dict)

In [7]:
press_release_details[0]

{'url': 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20210625a.htm',
 'date': 'June 25, 2021',
 'time': 'For release at 3:30 p.m. EDT'}

In [16]:
df = pd.DataFrame(press_release_details)
df.head()

,url,date,time
0,https://www.federalreserve.gov/newsevents/pres...,"June 25, 2021",For release at 3:30 p.m. EDT
1,https://www.federalreserve.gov/newsevents/pres...,"June 25, 2021",For release at 3:00 p.m. EDT
2,https://www.federalreserve.gov/newsevents/pres...,"June 24, 2021",For release at 4:30 p.m. EDT
3,https://www.federalreserve.gov/newsevents/pres...,"June 23, 2021",For release at 9:45 a.m. EDT
4,https://www.federalreserve.gov/newsevents/pres...,"June 22, 2021",For release at 10:30 a.m. EDT


In [24]:
df['time'] = df['time'].astype(str).str[15:]

KeyError: 'time'

,url,date,time
0,https://www.federalreserve.gov/newsevents/pres...,"June 25, 2021",3:30 p.m. EDT
1,https://www.federalreserve.gov/newsevents/pres...,"June 25, 2021",3:00 p.m. EDT
2,https://www.federalreserve.gov/newsevents/pres...,"June 24, 2021",4:30 p.m. EDT
3,https://www.federalreserve.gov/newsevents/pres...,"June 23, 2021",9:45 a.m. EDT
4,https://www.federalreserve.gov/newsevents/pres...,"June 22, 2021",10:30 a.m. EDT


In [19]:
df["datetime"] = df["date"] + " " + df["time"]

In [22]:
del df["date"]
del df["time"]

In [23]:
df.head()

,url,datetime
0,https://www.federalreserve.gov/newsevents/pres...,"June 25, 2021 3:30 p.m. EDT"
1,https://www.federalreserve.gov/newsevents/pres...,"June 25, 2021 3:00 p.m. EDT"
2,https://www.federalreserve.gov/newsevents/pres...,"June 24, 2021 4:30 p.m. EDT"
3,https://www.federalreserve.gov/newsevents/pres...,"June 23, 2021 9:45 a.m. EDT"
4,https://www.federalreserve.gov/newsevents/pres...,"June 22, 2021 10:30 a.m. EDT"
